In [ ]:
import os
import pickle

In [ ]:
!pip install langchain
!pip install langchain-core
!pip install langchain-community

In [ ]:
!pip install colab-xterm

In [ ]:
%load_ext colabxterm

Use the below commands to bring llm to our environment

curl -fsSL https://ollama.com/install.sh | sh 


ollama serve & ollama run llama2

In [ ]:
%xterm

In [ ]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [ ]:
!pip install unstructured
!pip install sentence-transformers
!pip install faiss-cpu
!pip install sentence-transformers

In [ ]:
import langchain
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.vectorstores import FAISS

In [ ]:
loaders = UnstructuredURLLoader(urls=[
    "<Link_to_Scribe>"
])
data = loaders.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
docs = text_splitter.split_documents(data)

In [ ]:
# Create the embeddings of the chunks using openAIEmbeddings
embeddings = HuggingFaceEmbeddings()

# Pass the documents and embeddings inorder to create FAISS vector index
vectorindex_openai = FAISS.from_documents(docs, embeddings)

In [ ]:
# Storing vector index create in local
file_path="vector_index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vectorindex_openai, f)

if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex = pickle.load(f)

In [ ]:
chain = RetrievalQA.from_chain_type(llm=llm,
                                 retriever=vectorIndex.as_retriever(),
                                 return_source_documents=True)

In [ ]:
query = "Read this site body, meta and cover other accessible data. Only this page I provided and no other subpages. Next, Print the title/name of the project, project description, Current status of the project, Stages, date in which this project information is published,  project Procurement/buying method, Budget of the project, Currency code (analyze from the page), buyer, Sector and Subsector in this format -> {1. title/name of the project, project description,Current status of the project,Stages,date in which this project information is published,project Procurement/buying method-Budget of the project,Currency code,buyer,Sector,Subsector.} | If any value is not found - mention \"not found\""

langchain.debug=True
result = chain({"query": query}, return_only_outputs=True)


In [ ]:
text = result['result']
answer_start = text.find("Title")

# Extract the text after "Answer:" until a blank line is found
answer_text = text[answer_start:text.find("\n\n", answer_start)]

# Print the extracted text
print(answer_text)
